In [1]:
import crewai
print(crewai.__version__)

0.130.0


In [2]:
import os
from crewai import Agent, Crew, Process, Task ,LLM
import requests

In [12]:
from crewai_tools import SerperDevTool ,ScrapeWebsiteTool, SnowflakeSearchTool, SnowflakeConfig

In [13]:
from datetime import date
from pydantic import BaseModel, Field
from pydantic import BaseModel, HttpUrl, Field
from datetime import date
from typing import List, Literal, Union
from decimal import Decimal


In [14]:
query = "SELECT DISTINCT SHIP_MANAGER \
    FROM CURATED_DB.ADDRESSABLE_MARKET.IHS_HISTORICAL_DATA_SHIP_MANAGER_TIMELINE \
    WHERE SHIP_MANAGER IS NOT NULL \
    ORDER BY SHIP_MANAGER"
# Create Snowflake configuration
config = SnowflakeConfig(
    account="SYNERGYGRP-DWH_DEV",
    user="SYNERGYGRP-DWH_DEV",
    password="Synergy@123",
    warehouse="COMPUTE_WH",
    database="CURATED_DB",
    snowflake_schema="ADDRESSABLE_MARKET",
	query=query
)

# Initialize the tool
snowflake_tool = SnowflakeSearchTool(config=config)


In [15]:
model='azure/gpt-4.1'
AZURE_API_KEY='C0AobTHxvT16LiGF5WN9x3nBgFcQK3B8NtcTVJLKyGMDlyZfzF0FJQQJ99BDACHYHv6XJ3w3AAAAACOGVWPp'
AZURE_API_BASE='https://azai-useast2-synergy.cognitiveservices.azure.com/'
AZURE_API_VERSION='2024-12-01-preview'
SERPER_API_KEY='d0540e84ba2200f8e2eb88bc81d3b3f489b523ff'

In [16]:
os.environ["AZURE_API_KEY"] = AZURE_API_KEY
os.environ["AZURE_API_BASE"] = AZURE_API_BASE
os.environ["AZURE_API_VERSION"] = AZURE_API_VERSION
os.environ["model"] = model

In [17]:
serper_api_key='d0540e84ba2200f8e2eb88bc81d3b3f489b523ff'
if not os.environ.get("SERPER_API_KEY"):
    os.environ["SERPER_API_KEY"] = serper_api_key

In [ ]:
lead_researcher = Agent(
    role="Lead Researcher",
    goal="Identify top ship owners in the specified {ship_class} and {region} who could benefit from Synergy Marine Group’s Crew & Technical Management services.Ships should have new builds",
    tools=[SerperDevTool(),ScrapeWebsiteTool(website_url="https://www.synergymarinegroup.com/services/offshore-ship-management-services/")],
    backstory="You work for Synergy Maritime - the second largest ship manager in the world - we have recently got into managing new segments of vessels through our EDT Joint Venture / Partnership Specializes in maritime industry research, focusing on specific market segments and geographical areas. Please use the tools [ScrapeWebsiteTool] available to you to get relevant working context.",
    verbose=True,
    allow_delegation=False,
    llm = 'azure/gpt-4.1',
)

In [18]:
review_agent = Agent(
    role="Review Specialist",
    goal="Ensure the list of identified leads is relevant and aligned with Synergy’s business model.",
    tools=[SerperDevTool(),snowflake_tool],
    backstory="You work for Synergy Maritime - we have recently got into managing new segments of vessels through our EDT Joint Venture / Partnership Responsible for maintaining the integrity of lead generation by filtering out existing partners and competitors.",
    verbose=True,
    allow_delegation=False,
    llm = 'azure/gpt-4.1',
)

In [19]:
strategy_developer = Agent(
    role="Strategy Developer",
    goal="Develop a personalized engagement strategy for each shortlisted ship owner, focusing on Crew & Technical Management.",
    tools=[SerperDevTool(),ScrapeWebsiteTool(website_url="https://www.synergymarinegroup.com/edt-synergy-marine-group-form-joint-venture/"),
		  			ScrapeWebsiteTool(website_url="https://www.edtoffshore.com/solutions-services/ship-management-operations")],
    backstory="You work for Synergy Maritime - the second largest ship manager in the world - we have recently got into managing new segments of vessels through our EDT Joint Venture / Partnership Crafts strategic sales plans based on comprehensive market research,Focusing on creating effective entry points for new business engagements.",
    verbose=True,
    allow_delegation=False,
    llm = 'azure/gpt-4.1',
)

In [20]:
process_manager = Agent(
    role="Process Manager",
    goal="Oversee the entire lead generation and strategy execution process, ensuring it aligns with Synergy Marine Group’s objectives.",
    tools=[SerperDevTool()],
    backstory="Coordinates and manages complex projects, ensuring that all components of the lead generation process are executed efficiently and in line with company objectives.",
    verbose=True,
    allow_delegation=False,
    llm = 'azure/gpt-4.1',
)

In [21]:


initial_research = Task(
    description="Conduct searches to identify relevant ship owners and operators based \
    on {ship_class} and {region}. Utilize WebsiteSearchTool and SerperDevTool for \
    comprehensive market analysis.Prioritize companies that are expanding, seeking professional ship management, or may require new technical or crew management solutions.Focus on ship owners with large fleets, new builds, or those looking to optimize operational efficiency.Gather essential company details (fleet size, operational focus, key decision-makers, and management structure).",
    expected_output="A list of potential ship owners, operators, and managers in the specified class and region. \
                  PLease review and rerun the task till you have 20-25 promising leads",
    agent=lead_researcher,
    async_execution=False,
    llm='azure/gpt-4.1',
)


In [23]:

lead_review = Task(
    description="Cross-check the extracted leads against the output list generated by the snowflake_tool to remove existing clients, and non-relevant prospects.Verify the regional presence and headquarters location of each company to confirm they align with {region}.Filter out companies that already have strong in-house ship management unless there’s potential for outsourcing.Validate data accuracy and remove redundant or low-potential leads. \
                  Please review and rerun the task till you have 20-25 promising leads",
    expected_output="A curated list of leads, excluding current partners focussed on EDT and similar partners \
                  PLease review and rerun the task till you have 20-25 promising leads",
    agent=lead_researcher,
    async_execution=False,
    llm='azure/gpt-4-1106-preview',
    context=[initial_research],
    tools=[SerperDevTool()],
)

In [24]:
strategy_formulation = Task(
    description="Determine alignment of the leads with Synergy’s goals and objectives. Create strategy by targeting the potential challenges the curated leads face (e.g., crew shortages, regulatory compliance, maintenance costs, etc.. ) and how Synergy’s services can help tackle them. Refine the strategy further by tailoring the pitch for different types of ship owners (independent, corporate, fleet operators, or newcomers). Define in-depth approach strategy by crafting a compelling pitch highlighting Synergy Marine Group’s strengths in crew management, technical expertise, cost efficiency, and compliance and  outline the best channels for engagement (cold outreach, industry events, partnerships, or referrals).",
    expected_output="A JSON object with the following fields Lead Name, Headquarters, Websites, Types of Vessels [with Number], Alignment, Strategy.The Strategy must be a JSON object with Pitch and Channels as fields. \
        Please review and rerun the task till you have 20-25 promising leads.",
    agent=strategy_developer,
    async_execution=False,
    llm= 'azure/gpt-4.1'
)

In [ ]:
strategy_formulation = Task(
    description="Craft a compelling pitch highlighting Synergy Marine Group's strengths in crew management, technical expertise, cost efficiency, and compliance. Tailor the approach for different types of ship owners (independent, corporate, fleet operators, or newcomers). Identify potential challenges they face (e.g., crew shortages, regulatory compliance, maintenance costs, etc.) and align solutions with Synergy's services. Outline the best channels for engagement (cold outreach, industry events, partnerships, or referrals).",
    expected_output="""{
        "strategy_report": {
            "status": "success",
            "leads": [...],
            "performance_metrics": {...},
            "metadata": {...}
        }
    }""",  # optional, can be retained for documentation
    agent=strategy_developer,
    async_execution=False,
    llm='azure/gpt-4.1',
    output_json=StrategyOutput,
    tools=[
        SerperDevTool(),
        ScrapeWebsiteTool(website_url="https://www.synergymarinegroup.com/edt-synergy-marine-group-form-joint-venture/"),
        ScrapeWebsiteTool(website_url="https://www.edtoffshore.com/solutions-services/ship-management-operations")
    ]
)


In [25]:
class strategy(BaseModel):
	"""Strategy model to store the strategy information"""
	pitch: str
	channels: str

class lead_data(BaseModel):
	"""Leads model to store the leads information"""
	lead_name: str
	headquarters: str
	website: str
	types_of_vessels: str
	alignment: str
	strategy: strategy

class lead(BaseModel):
	"""Lead model to store the lead information"""
	leads: List[lead_data] = Field(default_factory=list)


process_oversight = Task(
    description="Review and validate the final list of 10-15 high-potential leads before approval.Ensure all shortlisted leads align with Synergy’s growth strategy, ship management capabilities and core services(Crew & Technical Management).Iterate and refine the lead list and strategy until the most promising leads are identified. Please include headquarters region of the suggested lead and and have new builds, if its not in Northern Europe requested by user return back to lead_researcher to suggest an alternate.Ensure there are no duplicates within the leads data.Please include strategy for each lead and how it fits with synergys' goal.Ensure that the leads strategy is created from Synergy’s strategic business goals.Make your search extensive and order the leads in terms of appeal to synergy.",
    expected_output="A final list of 10-15 high-potential leads with each lead's json object.",
    agent=process_manager,
    async_execution=False,
    llm='azure/gpt-4.1',
    context=[lead_review, initial_research, strategy_formulation],
    output_json=lead
)


In [ ]:
ship_class = 'LNG'
region = 'Greece'
exclusion_list = ''
# Build and Run Crew
inputs = {
					'ship_class': ship_class,
					'region': region,
					'exclusion_list': ''
			}
# Build and Run Crew
crew = Crew(
	agents=[lead_researcher, review_agent, strategy_developer, process_manager],
	tasks=[initial_research, lead_review, strategy_formulation, process_oversight],
	process=Process.hierarchical,
	manager_llm= 'azure/gpt-4.1',
	verbose=True,
	full_output=False,
)

#existing_file_path = '/Workspace/Users/pranav.r@synergyship.com/Leads_Results.docx'
result = crew.kickoff(inputs=inputs);
if hasattr(result, 'raw'):
	# Access the raw output of the CrewOutput object
	print("Raw Output:")
	crew_output_raw = result.raw
	print(crew_output_raw)
if hasattr(result, 'json'):
	# If the output is in JSON format, access it accordingly
	# Raw output as a string or JSON
	print("JSON Output:")
	crew_output_json = result.json
	print(crew_output_json)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 73d5605a-06fb-4170-a401-343fd93c9807                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Agent: Lead Researcher

## Task: Conduct searches to identify relevant ship owners and operators based     on LNG and Greece. Utilize 
WebsiteSearchTool and SerperDevTool for     comprehensive market analysis.Prioritize companies that are expanding, 
seeking professional ship management, or may require new technical or crew management solutions.Focus on ship 
owners with large fleets, new builds, or those looking to optimize operational efficiency.Gather essential company 
details (fleet size, operational focus, key decision-makers, and management structure).

c:\Users\pranav.r\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\rich\live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

c:\Users\pranav.r\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\rich\live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

KeyboardInterrupt: 

In [65]:
result.tasks_output[3].output_format

<OutputFormat.JSON: 'json'>

In [70]:
result.tasks_output[3].json

'{"status": "needs_revision", "validated_leads": {"count": 10, "items": [{"name": "BW LPG", "headquarters": {"location": "Oslo, Norway (listed), Singapore (HQ)", "region": "Northern Europe", "matches_target_region": true, "regional_presence": ["Oslo", "Northern Europe", "Global"]}, "fleet": {"size": 53, "types": [{"type": "Very Large Gas Carriers (VLGCs) \\u2013 LPG", "count": 53}]}, "strategic_fit": {"score": 9, "strengths": ["World\\u2019s largest VLGC (LPG) fleet", "Aggressive fleet expansion and modernization", "Significant institutional/family ownership with long-term perspective", "Highly focused on sustainability and digitalization"], "service_matches": ["Crew Management", "Technical Management", "Newbuild Support", "Digital Operations"]}, "strategy": {"approach": "Target C-level executives via Oslo/Nor-Shipping/Gastech events, direct board introductions. Pitch Synergy\\u2019s proven digital-first management, decarbonization expertise, and global LPG/newbuild management.", "time

In [103]:
import json
leads_data = json.loads(result.json)  # result is a Pydantic model instance
leads=leads_data['validated_leads']['items']

In [104]:
leads

[{'name': 'BW LPG',
  'headquarters': {'location': 'Oslo, Norway (listed), Singapore (HQ)',
   'region': 'Northern Europe',
   'matches_target_region': True,
   'regional_presence': ['Oslo', 'Northern Europe', 'Global']},
  'fleet': {'size': 53,
   'types': [{'type': 'Very Large Gas Carriers (VLGCs) – LPG', 'count': 53}]},
  'strategic_fit': {'score': 9,
   'strengths': ['World’s largest VLGC (LPG) fleet',
    'Aggressive fleet expansion and modernization',
    'Significant institutional/family ownership with long-term perspective',
    'Highly focused on sustainability and digitalization'],
   'service_matches': ['Crew Management',
    'Technical Management',
    'Newbuild Support',
    'Digital Operations']},
  'strategy': {'approach': 'Target C-level executives via Oslo/Nor-Shipping/Gastech events, direct board introductions. Pitch Synergy’s proven digital-first management, decarbonization expertise, and global LPG/newbuild management.',
   'timeline': 'Initiate outreach Q2 2024; pr

In [71]:
owners_ids = {'Amine':22484151,'Prabhu':15645168,'Karthik':21576662,'Admin':22966424}

In [79]:
# Pipedrive integration
import requests
PIPEDRIVE_API_TOKEN = '7270c2d903c17226eef5e148fb214fba6c956a13'
owner_id = owners_ids['Karthik']
channel_id = 612

In [73]:
from datetime import datetime

# Get current date and time
current_time = datetime.now()

# Format time as ISO 8601 date
formatted_time = current_time.strftime("%Y-%m-%d")

formatted_time

'2025-06-11'

In [91]:
leads = result.json

In [86]:
def create_organization(lead):
    """Create a new organization in Pipedrive with address as a custom field"""
    organization = lead['name']
    headquarters = lead.get('headquarters', '')
    
    # First, check if the address custom field exists
    custom_fields = requests.get(
        'https://api.pipedrive.com/v1/organizationFields',
        params={'api_token': PIPEDRIVE_API_TOKEN}
    )
    
    address_field_id = None
    if custom_fields.status_code == 200:
        for field in custom_fields.json()['data']:
            if field['name'].lower() == 'address':
                address_field_id = field['key']
                break
    
    # Prepare organization data with custom field
    org_data = {
        'name': organization,
        'add_time': formatted_time,
    }
    
    # Add address if custom field exists
    if address_field_id:
        org_data[address_field_id] = headquarters
    
    # Make POST request to create organization
    response = requests.post(
        'https://api.pipedrive.com/v1/organizations',
        json=org_data,
        params={'api_token': PIPEDRIVE_API_TOKEN}
    )
    
    # Check if request was successful
    if response.status_code == 201:
        organization_id = response.json()['data']['id']
        print(f"Organization '{organization}' created successfully with ID: {organization_id}")
        return organization_id
    else:
        print(f"Error creating organization: {response.status_code}")
        print(response.text)
        return None

In [87]:
def organization_exists(organization):
	# Check if organization exists
	response = requests.get(
		'https://api.pipedrive.com/v1/organizations/find',
		params={'term': organization, 'api_token': PIPEDRIVE_API_TOKEN}
	)
	print(f"Checking if organization '{organization}' exists...",organization)
	if response.status_code == 200:
		data = response.json()
		if data['data']:
			print(f"Organization '{organization}' already exists")
			return data['data'][0]['id']  # Return the ID of the existing organization
		else:
			print(f"Organization '{organization}' does not exist")
			return None
	else:
		#print(f"Error checking organization: {response.status_code}")
		#print(response.text)
		return None

In [76]:
def create_note_content(content):
    try:
        # Validate required keys exist
        required_keys = ['name', 'headquarters', 'fleet', 'strategic_fit', 'strategy', 'link']
        for key in required_keys:
            if key not in content:
                raise KeyError(f"Missing required key: {key}")

        note_content = f"""# Company Overview: {content['name']}
        
📍 **Headquarters:**
{content['headquarters']}

🚢 **Fleet:**
{content['fleet']}

⭐ **Strategic Fit:**
{content['strategic_fit']}

📈 **Strategy:**
{content['strategy']}

🔗 **Reference Link:**
{content['link']}"""

        return note_content.strip()  # Remove trailing whitespace

    except Exception as e:
        print(f"Error creating note content: {str(e)}")
        return None

In [77]:
result.tasks_output[0].json_dict

{'status': 'success',
 'leads': [{'company_name': 'BW LPG',
   'website': 'https://www.bwlpg.com/',
   'fleet_size': 53,
   'ship_class_specialization': 'Very Large Gas Carriers (VLGCs) – LPG Shipping',
   'operational_region': 'Global, with Northern European presence (listed in Oslo, offices worldwide)',
   'recent_activity': 'Major fleet expansion in 2024 with acquisition of 12 VLGCs from Avance Gas, increasing total to 53; sustained newbuild deliveries and acquisitions; focus on decarbonization and technology-driven fleet renewal.',
   'management_needs': 'Likely to seek advanced technical management, crewing solutions, newbuild support, and digital operational efficiency, especially as the fleet grows with newer and more technologically advanced vessels.',
   'key_decision_makers': [{'name': 'Anders Onarheim',
     'position': 'Chief Executive Officer',
     'contact_info': None},
    {'name': 'Elaine Ong',
     'position': 'Chief Financial Officer',
     'contact_info': None},
   

In [79]:
for lead in leads:
	organization = lead['name']
	print(f"Processing organization: {organization}")
    
    # Check if organization exists
	org_id = organization_exists(organization)
    
    # If organization does not exist, create it
	if org_id is None:
		print(f"Create organization: {organization}")
		org_id = create_organization(lead)
    
	
	name = lead.get('name', 'No Name')
	headquarters = lead.get('company', 'No Company')
	fleet = lead.get('fleet', 'No Fleet')
	strategic_fit = lead.get('strategic_fit', 'No Fit')
	strategy = lead.get('strategy', 'No Strategy')
	link = lead.get('link', 'No Link')
	leads_data = {
		'title': name,
		'owner_id': owner_id,
		#'person_id': None,  # Assuming you don't have person_id
		'organization_id': org_id,
		#'expected_close_date': formatted_time,
		#'channel':channel_id,
	}

	# Make POST request to create lead
	response = requests.post('https://api.pipedrive.com/v1/leads',
		json=leads_data,
		params={'api_token': PIPEDRIVE_API_TOKEN}
	)
	if response.status_code == 201:  # 201 is success for POST
		print(f"Lead '{name}' created successfully")
		lead_id = response.json()['data']['id']
		print(f"Lead ID: {lead_id}")
	else:
		print(f"Error creating lead: {response.status_code}")
	

	# Create note content
	note_data = {
    'content': create_note_content(lead),
    'lead_id': lead_id,
    #'org_id': org_id,
    'add_time': formatted_time
	}

# Make POST request to create note
	response = requests.post(
	'https://api.pipedrive.com/v1/notes',
	json=note_data,
	params={'api_token': PIPEDRIVE_API_TOKEN}
	)

# Check if request was successful
	if response.status_code == 201:
		print("Successfully added company overview note")
		print(response.json())
	else:
		print(f"Error adding note: {response.status_code}")
		print(response.text)
	
	
    # Now you can use org_id for further operations like creating deals or persons
	#print(f"Organization ID for {organization}: {org_id}")

Processing organization: Tidewater Inc.
,Checking if organization 'Tidewater Inc.' exists... Tidewater Inc.
,Organization 'Tidewater Inc.' does not exist
,Create organization: Tidewater Inc.
,Organization 'Tidewater Inc.' created successfully with ID: 3854
,Lead 'Tidewater Inc.' created successfully
,Lead ID: 136e6d90-3d1f-11f0-8085-b3a2a921474b
,Successfully added company overview note
,{'success': True, 'data': {'id': 1022, 'user_id': 22966424, 'deal_id': None, 'person_id': None, 'org_id': 3854, 'lead_id': '136e6d90-3d1f-11f0-8085-b3a2a921474b', 'content': '# Company Overview: Tidewater Inc.<br />\n<br />\n📍 **Headquarters:**<br />\nHouston, Texas, USA (Northern Europe operations base)<br />\n<br />\n🚢 **Fleet:**<br />\nOver 200 vessels globally; 52 PSVs in Northern Europe after Solstad acquisition; 14 battery hybrid and 2 LNG PSVs in latest newbuild wave<br />\n<br />\n⭐ **Strategic Fit:**<br />\nRapidly expanded North Sea PSV fleet creates demand for crew/tech management scale, cos

In [ ]:

for lead in leads:
    organization = lead['name']
    print(f"Processing organization: {organization}")
    
    # Check if organization exists
    org_id = organization_exists(organization)
    
    # If organization does not exist, create it
    if org_id is None:
        print(f"Creating organization: {organization}")
        org_id = create_organization(lead)
        if org_id is None:
            print(f"Failed to create organization {organization}")
            continue
    
    # Prepare lead data
    leads_data = {
        'title': lead['name'],
        'owner_id': owner_id,
        'organization_id': org_id,
    }

    # Create lead
    response = requests.post(
        'https://api.pipedrive.com/v1/leads',
        json=leads_data,
        params={'api_token': PIPEDRIVE_API_TOKEN}
    )
    
    if response.status_code == 201:
        print(f"Lead '{lead['name']}' created successfully")
        lead_id = response.json()['data']['id']
        print(f"Lead ID: {lead_id}")
        
        # Create note
        note_data = {
            'content': create_note_content(lead),
            'lead_id': lead_id,
            'add_time': formatted_time
        }
        
        response = requests.post(
            'https://api.pipedrive.com/v1/notes',
            json=note_data,
            params={'api_token': PIPEDRIVE_API_TOKEN}
        )
        
        if response.status_code == 201:
            print("Successfully added company overview note")
        else:
            print(f"Error adding note: {response.status_code}")
            print(response.text)
    else:
        print(f"Error creating lead: {response.status_code}")
        print(response.text)

Processing organization: Odfjell SE
,Checking if organization 'Odfjell SE' exists... Odfjell SE
,Organization 'Odfjell SE' already exists
,Lead 'Odfjell SE' created successfully
,Lead ID: 96537020-3d1a-11f0-b421-276929a2ede6
,Successfully added company overview note
,Processing organization: Euronav NV
,Checking if organization 'Euronav NV' exists... Euronav NV
,Organization 'Euronav NV' does not exist
,Creating organization: Euronav NV
,Organization 'Euronav NV' created successfully
,Organization ID: 3844
,{'success': True, 'data': {'id': 3844, 'owner_id': {'id': 22966424, 'name': 'Data Team', 'email': 'spdatateam@synergyship.com', 'has_pic': 0, 'pic_hash': None, 'active_flag': True, 'value': 22966424}, 'name': 'Euronav NV', 'open_deals_count': 0, 'related_open_deals_count': 0, 'closed_deals_count': 0, 'related_closed_deals_count': 0, 'email_messages_count': 0, 'people_count': 0, 'activities_count': 0, 'done_activities_count': 0, 'undone_activities_count': 0, 'files_count': 0, 'notes_